In [43]:
import pandas as pd
import numpy as np
import sqlite3

import asyncio
import aiohttp
### Tohle vyřeší error ###
import nest_asyncio
nest_asyncio.apply()

In [44]:
def urls_from_indices(indices):
    urls=["https://www.sreality.cz/api/cs/v2/estates/"+str(i) for i in indices]
    return urls

async def get_response(session, url):
    try:
        async with session.get(url) as response:
            response_text =  await response.json() #parse response to json/ or response.text() for text
            #here you can process each further
            return response_text
    except aiohttp.ClientError as e:
        return f"Error occured for {url} : {e}"

async def main(urls, chunk_size):
    async with aiohttp.ClientSession() as session: 
        all_responses=[]
        chunks = [urls[i:i+chunk_size] for i in range(0, len(urls), chunk_size)]
        for chunk_idx, chunk in enumerate(chunks):
            print(f'running chunk {chunk_idx+1} out of {len(chunks)} chunks ({chunk_size} items)')
            #here you process first batch -> request go async

            tasks = [get_response(session, url) for url in chunk]
            #here they come together
            responses = await asyncio.gather(*tasks)
            
            #responses can be saved -> csv, sqlite etc
            #here we incorporate sqlite
            #to name each observation you could use: response["_embedded"]["favourite"]["_links"]["self"]["href"][17:]
            all_responses=all_responses+responses
        return all_responses # returns list

In [45]:
#loop = asyncio.get_event_loop()
#loop.run_until_complete(main(["https://run.mocky.io/v3/d8954f6b-054a-45f4-81a8-1541257f6c39"]*10, 2))

In [46]:
hash_ids = [3268089676,    #len(hash_ids) = 20
            1147786316,
            726730572,
            118862924,
            3608523852,
            2124076108,
            3885683788,
            1474597964,
            853942620,
            2797261900,
            3511018572,
            2504045644,
            636802892,
            2798949452,
            2019338060,
            1812649036,
            1611437132,
            2781501260,
            2564486220,
            1058464844]

urls=urls_from_indices(hash_ids)
loop = asyncio.get_event_loop()
output_list = loop.run_until_complete(main(urls, 5)) # len(output) = 20

running chunk 1 out of 4 chunks (5 items)


c:\Users\novot\anaconda3\lib\site-packages\charset_normalizer\api.py:105: UserWarning: Trying to detect encoding from a tiny portion of (20) byte(s).
  warn('Trying to detect encoding from a tiny portion of ({}) byte(s).'.format(length))


running chunk 2 out of 4 chunks (5 items)
running chunk 3 out of 4 chunks (5 items)
running chunk 4 out of 4 chunks (5 items)


In [47]:
# Getting rid of NaN rows
output_list = [i for i in output_list if i not in [item for item in output_list if len(item) == 1]]

In [48]:
items_dict={"Zlevněno":"discounted", 
"Původní cena":"price_original", 
"ID zakázky":"id_order", 
"Aktualizace":"date_update", 
"Stavba":"building_type", 
"Stav objektu":"building_condition", 
"Umístění objektu":"building_location", 
"Užitná plocha":"area_net", 
"Plocha podlahová":"area_floor", 
"Celková cena":"price_total", 
"Poznámka k ceně":"price_note", 
"Doprava":"transport", 
"Plocha zastavěná":"area_build_up", 
"Datum nastěhování":"date_moving_in", 
"Rok kolaudace":"date_building_approval", 
"Vybavení":"equipped", 
"Elektřina":"electricity", 
"Komunikace":"traffic_communication", 
"Bezbariérový":"barrier_free", 
"ID":"id", 
#"Cena":"price", 
"Energetická náročnost budovy":"energy_efficient_rating", 
"Výtah":"lift", 
"Typ domu":"building_floor_type", 
"Podlaží":"building_floor", 
"Voda":"water", 
"Plyn":"gas", 
"Odpad":"waste", 
"Topení":"heating", 
"Telekomunikace":"telecommunication", 
"Rok rekonstrukce":"date_year_reconstruction", 
"Průkaz energetické náročnosti budovy":"energy_efficient_rating_card", 
"Stav":"availability", 
"Datum zahájení prodeje":"date_offer_start", 
"Ukazatel energetické náročnosti budovy":"energy_efficient_rating_index", 
"Datum prohlídky":"date_tour", 
"Datum prohlídky do":"date_tour_to", 
"Plocha zahrady":"area_garden",
"Výška stropu":"ceiling_height",
"Parkování":"parking",
"Cena za m²":"price_sqm",
"Počet kanceláří":"office_count",
"Garáž":"garage",
"Počet míst":"place_count",
"Bazén":"basin",
"Plocha pozemku":"area_landplot",
"Velikost podílu":"share_size",
"Vyvolávací cena":"auction_initial_price",
"Znalecký posudek":"expert_opinion_official",
"Minimální příhoz":"auction_min_bid", 
"Aukční jistina":"auction_principal",
"Druh dražby":"auction_type",
"Místo konání dražby":"auction_place",
"Datum konání dražby":"auction_date",
"Dražební vyhláška":"auction_decree",
"Posudek znalce":"expert_opinion"}

def note_missing_values(r_dict_names_all):
        print("Add these values to your dictionary:")
        print(r_dict_names_all[~r_dict_names_all.isin(items_dict.keys())])

def individual_description_into_pd_df(description_individual):
        df_desc = pd.concat(description_individual).unstack()
        df_desc["equipped"]=df_desc["equipped"].map({True: "ano", False: "ne", "Částečně":"částečně"}) 
        df_desc.reset_index(inplace=True)
        df_desc = df_desc.rename(columns = {'index':'hash_id'})
        return df_desc

def description_decoding(responses_list):
        description_individual = {}
        r_dict_names_all=pd.Series(dtype="object")
        r_dict_types_all=pd.DataFrame(columns=["name", "type"])

        for r_dict in responses_list:
                
                info_relevant = pd.Series(dtype="object")
                info_relevant["description"]=r_dict["text"]["value"]

                r_dict_values=pd.DataFrame(r_dict["items"], columns =['type', 'name', 'value'])

                r_dict_names=r_dict_values["name"]
                r_dict_names_all=pd.concat([r_dict_names_all, r_dict_names[~r_dict_names.isin(r_dict_names_all)]])

                r_dict_types=r_dict_values[["type", "name"]]
                r_dict_types_all=pd.concat([r_dict_types_all, r_dict_types.loc[~r_dict_types["name"].isin(r_dict_types_all["name"]),:]])

                for name_raw in items_dict.keys():

                        name_clean=items_dict[name_raw]

                        if name_raw not in r_dict_names.values:
                                info_relevant[name_clean]=np.nan
                        elif r_dict_values[r_dict_names==name_raw]["type"].all()=="set":
                                index_nr=int(r_dict_values.index[r_dict_values['name'] == name_raw].tolist()[0])
                                info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw][index_nr][0]["value"].values[0]
                                del index_nr
                        else:
                                info_relevant[name_clean]=r_dict_values["value"][r_dict_values["name"]==name_raw].values[0]

                description_individual[r_dict["_embedded"]["favourite"]["_links"]["self"]["href"][17:]] = info_relevant
        
        note_missing_values(r_dict_names_all)

        df_final=individual_description_into_pd_df(description_individual)

        return df_final

df = description_decoding(output_list)
df

Add these values to your dictionary:
6            Poloha domu
10                 Sklep
1     Náklady na bydlení
13         Plocha bazénu
0                   Cena
dtype: object


,hash_id,description,discounted,price_original,id_order,date_update,building_type,building_condition,building_location,area_net,...,share_size,auction_initial_price,expert_opinion_official,auction_min_bid,auction_principal,auction_type,auction_place,auction_date,auction_decree,expert_opinion
0,3268089676,Energeticky nenáročný moderní dům se dvěma par...,NaN,NaN,128538,Dnes,Smíšená,Novostavba,NaN,155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,726730572,Unikátní prvorepubliková vila v blízkosti cent...,NaN,NaN,128637,Dnes,Cihlová,Velmi dobrý,NaN,490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118862924,K dlouhodobému podnájmu nabízíme právě dokonče...,NaN,NaN,N3310,Dnes,Cihlová,Novostavba,Klidná část obce,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3608523852,Prohlídky\r\nPro registraci na prohlídku použi...,NaN,NaN,N3309,Dnes,Cihlová,Novostavba,Klidná část obce,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2124076108,"Horní Počernice, Zámezí, samostatný rodinný dů...",NaN,NaN,R4E-004159,Dnes,Cihlová,Po rekonstrukci,Klidná část obce,92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3885683788,Nabízíme k pronájmu hezký rodinný dům s dispoz...,NaN,NaN,2906,Dnes,Cihlová,Velmi dobrý,Klidná část obce,330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1474597964,Zcela unikátní a samostatně stojící vila s udr...,NaN,NaN,1535-1,Dnes,Cihlová,Velmi dobrý,NaN,410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,853942620,Pronájem luxusní vily v Praze 6 - Nebušicích. ...,NaN,NaN,NaN,18.01.2023,Cihlová,Velmi dobrý,Klidná část obce,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2797261900,"Nabízíme k pronájmu krásný, světlý rohový řado...",NaN,NaN,N660226,19.01.2023,Cihlová,Velmi dobrý,NaN,163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3511018572,"Pronájem řadového rodinného domu 6+1, o ploše ...",NaN,NaN,8677,16.01.2023,Panelová,Velmi dobrý,Klidná část obce,250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### **SQLite**

In [49]:
con = sqlite3.connect('Estate_data.sqlite') # We must choose the name for our DB !

In [50]:
# Creates a table or appends if exists
df.to_sql(name = 'DESCRIPTION_TABLE', con = con, index = False, if_exists = 'append') # TO_DO: hází error - asi nepodporuje všechny data typy

InterfaceError: Error binding parameter 29 - probably unsupported type.

In [51]:
# Closing the connection
con.close()

# Loading again: df = pd.read_sql('SELECT * FROM DESCRIPTION_TABLE', con = con)